In [1]:
# !wget https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
# !unzip hetrec2011-lastfm-2k.zip

--2024-04-23 12:51:08--  https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589075 (2.5M) [application/zip]
Saving to: ‘hetrec2011-lastfm-2k.zip’

hetrec2011-lastfm-2 100%[===================>]   2.47M  1.68MB/s    in 1.5s    

2024-04-23 12:51:11 (1.68 MB/s) - ‘hetrec2011-lastfm-2k.zip’ saved [2589075/2589075]

Archive:  hetrec2011-lastfm-2k.zip
  inflating: user_friends.dat        
  inflating: user_taggedartists.dat  
  inflating: user_taggedartists-timestamps.dat  
  inflating: artists.dat             
  inflating: readme.txt              
  inflating: tags.dat                
  inflating: user_artists.dat        


In [3]:
!head artists.dat

id	name	url	pictureURL
1	MALICE MIZER	http://www.last.fm/music/MALICE+MIZER	http://userserve-ak.last.fm/serve/252/10808.jpg
2	Diary of Dreams	http://www.last.fm/music/Diary+of+Dreams	http://userserve-ak.last.fm/serve/252/3052066.jpg
3	Carpathian Forest	http://www.last.fm/music/Carpathian+Forest	http://userserve-ak.last.fm/serve/252/40222717.jpg
4	Moi dix Mois	http://www.last.fm/music/Moi+dix+Mois	http://userserve-ak.last.fm/serve/252/54697835.png
5	Bella Morte	http://www.last.fm/music/Bella+Morte	http://userserve-ak.last.fm/serve/252/14789013.jpg
6	Moonspell	http://www.last.fm/music/Moonspell	http://userserve-ak.last.fm/serve/252/2181591.jpg
7	Marilyn Manson	http://www.last.fm/music/Marilyn+Manson	http://userserve-ak.last.fm/serve/252/2558217.jpg
8	DIR EN GREY	http://www.last.fm/music/DIR+EN+GREY	http://userserve-ak.last.fm/serve/252/46968835.png
9	Combichrist	http://www.last.fm/music/Combichrist	http://userserve-ak.last.fm/serve/252/51273485.jpg


In [5]:
artists = {}

with open("artists.dat", "r") as f:
    print(f.readline())
    
    for line in f:
        id, name, _, _ = line.split('\t')
        artists[int(id)] = name

id	name	url	pictureURL



In [6]:
!head user_artists.dat

userID	artistID	weight
2	51	13883
2	52	11690
2	53	11351
2	54	10300
2	55	8983
2	56	6152
2	57	5955
2	58	4616
2	59	4337


In [ ]:
import numpy as np

users = []
items = []
weights = []

with open("user_artists.dat", "r") as f:
    print(f.readline())

    for line in f:
        uid, aid, count = line.split('\t')

        users.append(int(uid))
        items.append(int(aid))
        weights.append(np.log(1+float(count)))

users = np.array(users)
items = np.array(items)
weights = np.array(weights)

userID	artistID	weight



In [27]:
from scipy.sparse import csc_matrix
from sklearn.linear_model import ElasticNet
from tqdm import tqdm

train = csc_matrix( (weights, (users, items)) )

model = ElasticNet(alpha=0.1, l1_ratio=0.5, positive=True, fit_intercept=False, copy_X=False, selection='random', tol=1e-6, max_iter=100)

rows = []
cols = []
data = []

for current_item in tqdm(range(max(items) + 1)):

    # 훈련할 값 (모든 row 값과 현재 item 번호에 해당하는 값들을 가져온다)
    y = train[:, current_item].toarray()

    # 만약 가져온 아이템이 전부 0이면 학습할 것이 없기 때문에 예외 (즉, 아이템에 어떤 평가도 존재하지 않음)
    if all(y==0): continue

    # 현재 item 값에 있는 값을 모두 0으로 만들어야 한다.
    s = train.indptr[current_item]
    e = train.indptr[current_item+1]
    backup = train.data[s:e].copy()
    train.data[s:e] = 0.0

    model.fit(train, y)

    rows.extend(model.sparse_coef_.indices)
    cols.extend([current_item] * model.sparse_coef_.getnnz())
    data.extend(model.sparse_coef_.data)
    
    train.data[s:e] = backup

# W matrix 생성
W = csc_matrix( (data, (rows, cols)), shape=(max(items) + 1, max(items) + 1) )

100%|██████████| 18746/18746 [00:43<00:00, 429.64it/s]


In [40]:
korea_artists = [1918, 2117, 2094, 1904]

for artists_number in korea_artists:
    s = W.indptr[artists_number]
    e = W.indptr[artists_number+1]
    
    artists_ids = W.indices[s:e]
    scores = W.data[s:e]

    print(f"{artists[artists_number]}와 유사한 artists:")

    for score, id in sorted(zip(scores, artists_ids), reverse=True)[:4]:
        print(f"{artists[id]}: {score}")

동방신기와 유사한 artists:
소녀시대: 0.1077790560626788
Super Junior: 0.09631992484775728
BoA: 0.023375248033484612
Brown Eyed Girls: 0.014559733566562752
시크릿와 유사한 artists:
4minute: 0.03359560033614966
B2ST: 0.024008900514853267
소녀시대: 0.018618586053677486
BoA: 0.002119523751437404
BoA와 유사한 artists:
倖田來未: 0.23389080651709723
2NE1: 0.188691589715216
소녀시대: 0.17656141785217552
SHINee: 0.10302762323104384
SHINee와 유사한 artists:
BIG BANG: 0.30146418608131753
티아라: 0.2049095659014247
U-KISS: 0.13088060613392763
4minute: 0.09929500632584676


In [48]:
users = [312, 43]

for uid in users:
    userdata = train[uid, :].tocsr()

    scores = userdata.dot(W)

    order = scores.data.argsort()[::-1]

    print(f"{uid}에게 추천할 artists:")
    for score, id in sorted(zip(scores.data, scores.indices), reverse=True)[:10]:
        if id not in userdata.indices:
            print(f"{artists[id]}: {score}")

312에게 추천할 artists:
宇多田ヒカル: 3.6422403584852043
SHINee: 2.969102873192674
43에게 추천할 artists:
